In [34]:
# magic command %memit 를 사용하기 위해 필요한 라이브러리, 
import memory_profiler 
%load_ext memory_profiler
# ipython_memory를 사용하여 cell별로 메모리를 계속 관찰하기 위한 라이브러리. 
import ipython_memory_usage 
import ipython_memory_usage.ipython_memory_usage as imu
%ipython_memory_usage_start

'memory profile enabled'

In [34] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 177.99 MiB


# Chap 5. 이터레이터와 제너레이터

### 이 장에서 배울 내용  

* 제너레이터에서 메모리를 절약하는 방법  
* 제너레이터가 유용한 상황
* 복잡한 제너레이터 작업에 itertools를 사용하는 방법
* 지연 연산이 효과적인 경우와 그렇지 않은 경우 

In [ ]:
# 파이썬의 for loop

for i in range(N):
    do_work(i)

In [ ]:
# 다른 언어의 for loop
for (i=0; i<N; i++) {
    do_work(i)
}

제너레이터 를 사용하면 이런 종류의 함수를 필요할 때마다 지연 계산 lazy evaluation 할 수 있어서  
성능 상에 영향을 주지 않고도 높은 가독성을 얻을 수 있다. 

In [26]:
# 여러 피보나치 수를 계산하는 함수 리스트 VS 제너레이터

def fibonacci_list(num_items):
    numbers = []
    a, b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a + b
    return numbers


def fibonacci_gen(num_items):
    a, b = 0, 1 
    while num_items:
        yield a
        a, b = b, a + b
        num_items -= 1

fibonacci list는 원하는 개수의 모든 피보나치 수를 담는 리스트를 생성하도록 구현  
1만개면, numbers 리스트에 값을 1만번 추가한 다음 리스트를 반환한다.  -> 부하가 발생한다.  

제너레이터는 코드가 yield를 실행하는 순간 이 함수는 그 값을 방출하고 ,   
다른 값 요청이 들어오면 이전 상태를 유지한 채로 실행을 재개하여 새로운 값을 방출한다.   

### fibonacci_list를 사용한 루프는 메모리를 1만 배 이상 더 사용한다.  

파이썬의 for loop 에는 반복할 수 있는 객체가 필요함 = 루프 밖에서 이터레이터를 생성할 수 있어야 함.  
파이썬의 대부분의 객체에서 이터레이터를 생성하려면 파이썬의 내장함수 __iter__ffmf tkdyd  -> 리스트, 튜플, 딕셔너리, 셋 에서 사용 가능

### 파이썬의 for문은 내부적으로 iterator를 생성하여 동작한다. __iter__ method를 이용하여.   
만약 리스트를 순회하는 for문이라면, 해당 리스트의 iterator를 생성하여  __next__ method를 이용해서 순회를 한다.  

### ITERABLE, ITERATOR   

내부 요소를 하나씩 리턴할 수 있는 객체 : iterable 하다.   ex) for 문을 통해 순회할 수 있는 객체  (시퀀스 타입, 컬렉션 타입)  
iterable : __next__ 가 존재 하지 않음  
iterator : __next__ 가 존재함

In [7]:
# iterable 객체를 iterator로 만들기
a = [1, 2, 3]
print(type(a))
a = iter(a)
print(type(a))

<class 'list'>
<class 'list_iterator'>


In [3]:
for i in iter([1, 2, 3, 4, 5]):
    print(i)

1
2
3
4
5


In [4]:
for i in [1, 2, 3, 4, 5]:
    print(i)

1
2
3
4
5


In [9]:
# 예제 5-1 파이썬 for 루프 재구성하기 

for i in object:
    do_work(i)


object_iterator = iter(object)
while True:
    try:
        i = next(object_iterator)
    except StopIteration:
        break
    else:
        do_work(i)

TypeError: 'type' object is not iterable

fibonacci_gen 을 사용하면 이터레이터로 변형되는 제너레이터를 생성한다.  
하지만 fibonacci_list 는 새로운 리스트를 할당하고 값을 미리 계산한 다음에 이터레이터를 생성해야 한다.  

### 한 번에 값이 하나만 필요하더라도 fibonacci_list 리스트를 미리 계산하려면 전체 데이터를 저장할 수 있는 공간을 할당하고 올바른 값을 넣어야 함  
### fibonacci_list는 사용할 수 있는 용량 보다 더 많은 메모리 할당을 시도해서,  루프 자체를 실행하지 못할 수도 있다.  
### fibonacci_list(100_000_000) dms 3.1 GB 크기의 리스트를 생성한다.  

In [27]:
def test_fibonacci_list():
    """
    >>> a
    
    >>> 
    """

    for i in fibonacci_list(100_000):
        pass

In [28]:
%timeit test_fibonacci_list()

433 ms ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%memit test_fibonacci_list()

peak memory: 503.32 MiB, increment: 325.31 MiB
In [35] used 43.1797 MiB RAM in 1.49s, peaked 284.75 MiB above current, total RAM usage 221.17 MiB


In [30]:
def test_fibonacci_gen():

    for i in fibonacci_gen(100_000):
        pass

In [31]:
%timeit test_fibonacci_gen()

129 ms ± 757 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%memit test_fibonacci_gen()

peak memory: 205.18 MiB, increment: -7.99 MiB
In [36] used -15.9883 MiB RAM in 1.63s, peaked 15.99 MiB above current, total RAM usage 205.18 MiB


제너레이터 버전이 두 배 더 빠르고 메모리도 적게 사용한다.  

In [37]:
divisible_by_three = len([n for n in fibonacci_gen(100_000) if n% 3 ==0])

In [37] used 18.7031 MiB RAM in 1.72s, peaked 0.00 MiB above current, total RAM usage 223.88 MiB
